In [163]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize 
import plotly.express as px
# input_file = open('/opendota api constants/heroes.json')
# hero_id = json.load(input_file)
# input_file.close()
# print('hero_id')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### Loading Data 

Import json files and convert to dataframe

** better to use json.load or pandas.read_json?

In [164]:
### cleaning league x data 
#matches/league_id_10749_ti9_matchdata.json
#matches/league_id_10979_starladder_s2_matchdata.json
#matches/league_id_10826_epicenter_2019_matchdata.json
#matches/league_id_11280_chengdu_matchdata.json
#matches/league_id_11371_summit_11_matchdata.json

input_file = open('matches/league_id_10979_starladder_s2_matchdata.json') 
data = json.load(input_file)
input_file.close()

In [165]:
big_df = json_normalize(data)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



### Create Dataframe for match results

This will cover general id columns and basic match results such as match id, league id, dire/radiant teams, and if radiant won

Method 1: since data is saved as json, use pd.read_json

In [166]:
#data2 = pd.read_json('matches/league_id_10749_ti9_matchdata.json')
#data2.head(1)

In [167]:
match_df = big_df[['match_id','duration', 'leagueid','match_seq_num','series_id', 'series_type', 'patch', 'dire_team_id', 'radiant_team_id', 'radiant_win' ]]
match_df.shape
#match_df.isnull

(153, 10)

In [168]:
#match_df.index('match_id')

In [169]:
#match_df.head()

In [170]:
#match_df.sort_values(by='match_id', ascending=True)

## Method 2: load json file and normalize using pandas.json_normalize.

** Dunno if there is any significant advantage to doing it one way or the other. 

In [171]:
#json_struct = json.loads(data.to_json(orient="records"))    
#df_flat = pd.io.json.json_normalize(json_struct) #use pd.io.json

In [172]:
picks_df = json_normalize(data, 'picks_bans', max_level=0)
#picks_df.head(100)
#picks_df.shape

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



In [173]:
df = picks_df[['match_id', 'order', 'is_pick', 'team', 'hero_id']]

In [174]:
df = df.astype(str)
df = df.groupby('match_id')['is_pick','team','hero_id'].agg(' '.join)
#df.head(100)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [175]:
#new = df['order'].str.split(' ', expand=True)
new1 = df['is_pick'].str.split(' ', expand=True)
new2 = df['team'].str.split(' ', expand=True)
new3 = df['hero_id'].str.split(' ', expand=True)
#new1, new2, new3

In [176]:
#nameless_df= new1.join(new2).join(new3)

In [177]:
# Get column labels for the columns 
total_picks = 21 # recall range is 0-21 = 22 picks
new1.columns = ['1is_pick', '2is_pick', '3is_pick', '4is_pick', '5is_pick', '6is_pick', '7is_pick','8is_pick','9is_pick','10is_pick','11is_pick','12is_pick','13is_pick','14is_pick','15is_pick','16is_pick','17is_pick','18is_pick','19is_pick','20is_pick','21is_pick', '22is_pick']
new2.columns = ['1team', '2team', '3team', '4team', '5team', '6team', '7team', '8team', '9team', '10team', '11team', '12team', '13team', '14team', '15team', '16team', '17team', '18team', '19team', '20team', '21team', '22team']
new3.columns = ['1hero_id', '2hero_id', '3hero_id', '4hero_id', '5hero_id', '6hero_id', '7hero_id', '8hero_id', '9hero_id', '10hero_id', '11hero_id', '12hero_id', '13hero_id', '14hero_id', '15hero_id', '16hero_id', '17hero_id', '18hero_id', '19hero_id', '20hero_id', '21hero_id',
               '22hero_id']

In [178]:
data = new1.join(new2).join(new3)
#picks_df = new1.join(new2).join(new3)
#picks_df.dtypes
#picks_df.head(100)
#data.shape

Quick eyeball inspection to check if the draft information is preserved:


Comparing match_id = 4888646940, we can see from above that the first pick (order=0) information was: 
- first pick (1ban): '4888646940	0	False	1	66'; w/c lines up with 1is_pick, 1hero_id, etc...
- last pick: 9106	4888646940	21	True	0	67; w/c lines up with 22is_pick, 22hero_id, etc...

So, at least by a quick visual inspection, seems like the drafting data has been successfully cleaned. 

### Replacing T/F values

Convert T/F to boolean 

In [179]:
# source: https://github.com/codexponent/dota2-draft-analysis/blob/master/draft-ml.ipynb
# # # Converting boolean value into number
#boolean_values = 22
#for i in range(boolean_values):
#    pick_id = '{}is_pick'.format(i+1)
#    data[pick_id] = data[pick_id].apply(lambda x: 1 if x else 0)

Re-organize columns according to prefix order:
- pretty sure there's a better way to do this 

In [162]:
picks_df = data[['1is_pick', '1team', '1hero_id',
               '2is_pick', '2team', '2hero_id', 
               '3is_pick', '3team', '3hero_id', 
               '4is_pick', '4team', '4hero_id', 
               '5is_pick', '5team', '5hero_id', 
               '6is_pick', '6team', '6hero_id', 
               '7is_pick', '7team', '7hero_id', 
               '8is_pick', '8team', '8hero_id', 
               '9is_pick', '9team', '9hero_id', 
               '10is_pick', '10team', '10hero_id', 
               '11is_pick', '11team', '11hero_id', 
               '12is_pick', '12team', '12hero_id',
               '13is_pick', '13team', '13hero_id', 
               '14is_pick', '14team', '14hero_id', 
               '15is_pick', '15team', '15hero_id', 
               '16is_pick', '16team', '16hero_id', 
               '17is_pick', '17team', '17hero_id',
               '18is_pick', '18team', '18hero_id', 
               '19is_pick', '19team', '19hero_id',
               '20is_pick', '20team', '20hero_id',
               '21is_pick', '21team', '21hero_id', 
                '22is_pick', '22team', '22hero_id']]

In [189]:
picks.shape

(151, 66)

In [109]:
picks = picks_df
matchhistory = match_df

In [141]:
picks.to_csv('data/ti9_picks_cleaned.csv')
#matchhistory.to_csv('data/summit11_matchhistory.csv') 

In [188]:
picks.index

KeyError: 'match_id'

In [186]:
matchhistory

,match_id,duration,leagueid,match_seq_num,series_id,series_type,patch,dire_team_id,radiant_team_id,radiant_win
0,4833593109,4036,10979,4062972494,337111,1,41,7118032,111474,True
1,4774328947,3221,10979,4023489932,331091,1,40,7118032,6363102,False
2,4779407768,2568,10979,4026694023,331560,1,40,7019882,111474,True
3,4833371463,2976,10979,4062797533,337111,1,41,111474,7118032,True
4,4833120790,2700,10979,4062637540,337111,1,41,111474,7118032,False
5,4835973421,3504,10979,4064617551,337330,1,41,6214973,7059982,False
6,4835092211,2530,10979,4063983045,337299,1,41,7137328,7118032,True
7,4778183525,2956,10979,4025875326,331453,1,40,7137328,5055770,True
8,4835349235,2983,10979,4064165158,337299,1,41,7137328,7118032,False
9,4780631882,2250,10979,4027495162,331700,1,40,5327206,7137328,False


In [182]:
picks.head()

,1is_pick,1team,1hero_id,2is_pick,2team,2hero_id,3is_pick,3team,3hero_id,4is_pick,4team,4hero_id,5is_pick,5team,5hero_id,6is_pick,6team,6hero_id,7is_pick,7team,7hero_id,8is_pick,8team,8hero_id,9is_pick,9team,9hero_id,10is_pick,10team,10hero_id,11is_pick,11team,11hero_id,12is_pick,12team,12hero_id,13is_pick,13team,13hero_id,14is_pick,14team,14hero_id,15is_pick,15team,15hero_id,16is_pick,16team,16hero_id,17is_pick,17team,17hero_id,18is_pick,18team,18hero_id,19is_pick,19team,19hero_id,20is_pick,20team,20hero_id,21is_pick,21team,21hero_id,22is_pick,22team,22hero_id
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4770332364,True,True,33,True,True,55,True,True,94,True,True,102,True,True,53,True,True,88,True,True,69,True,True,6,True,True,3,True,True,87,True,True,36,True,True,77,True,True,65,True,True,18,True,True,16,True,True,52,True,True,25,True,True,46,True,True,29,True,True,50,True,True,7,True,True,17
4770343012,True,True,53,True,True,88,True,True,16,True,True,111,True,True,69,True,True,55,True,True,102,True,True,6,True,True,100,True,True,65,True,True,29,True,True,89,True,True,13,True,True,46,True,True,86,True,True,7,True,True,57,True,True,10,True,True,39,True,True,76,True,True,106,True,True,36
4770458746,True,True,6,True,True,55,True,True,53,True,True,88,True,True,52,True,True,102,True,True,69,True,True,111,True,True,95,True,True,15,True,True,65,True,True,18,True,True,33,True,True,112,True,True,16,True,True,86,True,True,25,True,True,87,True,True,94,True,True,49,True,True,39,True,True,10
4770490885,True,True,6,True,True,88,True,True,53,True,True,102,True,True,15,True,True,111,True,True,55,True,True,69,True,True,16,True,True,107,True,True,18,True,True,54,True,True,8,True,True,89,True,True,87,True,True,95,True,True,42,True,True,64,True,True,46,True,True,35,True,True,94,True,True,65
4770544546,True,True,94,True,True,88,True,True,69,True,True,55,True,True,53,True,True,25,True,True,102,True,True,111,True,True,19,True,True,95,True,True,76,True,True,27,True,True,15,True,True,45,True,True,52,True,True,16,True,True,17,True,True,39,True,True,29,True,True,6,True,True,60,True,True,89


In [183]:
matchhistory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   match_id         153 non-null    int64
 1   duration         153 non-null    int64
 2   leagueid         153 non-null    int64
 3   match_seq_num    153 non-null    int64
 4   series_id        153 non-null    int64
 5   series_type      153 non-null    int64
 6   patch            153 non-null    int64
 7   dire_team_id     153 non-null    int64
 8   radiant_team_id  153 non-null    int64
 9   radiant_win      153 non-null    bool 
dtypes: bool(1), int64(9)
memory usage: 11.0 KB


In [184]:
picks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 151 entries, 4770332364 to 4844547955
Data columns (total 66 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   1is_pick   151 non-null    bool 
 1   1team      151 non-null    bool 
 2   1hero_id   151 non-null    int64
 3   2is_pick   151 non-null    bool 
 4   2team      151 non-null    bool 
 5   2hero_id   151 non-null    int64
 6   3is_pick   151 non-null    bool 
 7   3team      151 non-null    bool 
 8   3hero_id   151 non-null    int64
 9   4is_pick   151 non-null    bool 
 10  4team      151 non-null    bool 
 11  4hero_id   151 non-null    int64
 12  5is_pick   151 non-null    bool 
 13  5team      151 non-null    bool 
 14  5hero_id   151 non-null    int64
 15  6is_pick   151 non-null    bool 
 16  6team      151 non-null    bool 
 17  6hero_id   151 non-null    int64
 18  7is_pick   151 non-null    bool 
 19  7team      151 non-null    bool 
 20  7hero_id   151 non-null    int64
 21  8is_p

In [185]:
# picks dictionary
picks_dict = {'1is_pick': bool, '1team': bool, '1hero_id': int,
               '2is_pick': bool, '2team': bool, '2hero_id': int, 
               '3is_pick': bool, '3team': bool, '3hero_id': int, 
               '4is_pick': bool, '4team': bool, '4hero_id': int, 
               '5is_pick': bool, '5team': bool, '5hero_id': int, 
               '6is_pick': bool, '6team': bool, '6hero_id': int, 
               '7is_pick': bool, '7team': bool, '7hero_id': int, 
               '8is_pick': bool, '8team': bool, '8hero_id': int, 
               '9is_pick': bool, '9team': bool, '9hero_id': int, 
               '10is_pick': bool, '10team':bool, '10hero_id': int, 
               '11is_pick': bool, '11team':bool, '11hero_id': int, 
               '12is_pick': bool, '12team':bool, '12hero_id': int,
               '13is_pick': bool, '13team':bool, '13hero_id': int, 
               '14is_pick': bool, '14team':bool, '14hero_id': int, 
               '15is_pick': bool, '15team':bool, '15hero_id': int, 
               '16is_pick': bool, '16team':bool, '16hero_id': int, 
               '17is_pick': bool, '17team':bool, '17hero_id': int,
               '18is_pick': bool, '18team':bool, '18hero_id': int, 
               '19is_pick': bool, '19team':bool, '19hero_id': int,
               '20is_pick': bool, '20team':bool, '20hero_id': int,
               '21is_pick': bool, '21team':bool, '21hero_id': int, 
                '22is_pick': bool, '22team':bool, '22hero_id': int}

picks = picks.astype(picks_dict)

In [143]:
picks.to_csv('data/starladder_picks_cleaned.csv')
matchhistory.to_csv('data/starladder_matches_cleaned.csv') 

In [144]:
picks2.dtypes

match_id     int64
1is_pick      bool
1team         bool
1hero_id     int64
2is_pick      bool
2team         bool
2hero_id     int64
3is_pick      bool
3team         bool
3hero_id     int64
4is_pick      bool
4team         bool
4hero_id     int64
5is_pick      bool
5team         bool
5hero_id     int64
6is_pick      bool
6team         bool
6hero_id     int64
7is_pick      bool
7team         bool
7hero_id     int64
8is_pick      bool
8team         bool
8hero_id     int64
9is_pick      bool
9team         bool
9hero_id     int64
10is_pick     bool
10team        bool
10hero_id    int64
11is_pick     bool
11team        bool
11hero_id    int64
12is_pick     bool
12team        bool
12hero_id    int64
13is_pick     bool
13team        bool
13hero_id    int64
14is_pick     bool
14team        bool
14hero_id    int64
15is_pick     bool
15team        bool
15hero_id    int64
16is_pick     bool
16team        bool
16hero_id    int64
17is_pick     bool
17team        bool
17hero_id    int64
18is_pick   